In [ ]:
import gzip
import json
import os
import pprint
import subprocess
import time
from datetime import timedelta, datetime
from pathlib import Path
from dataclasses import dataclass, asdict
import multiprocessing

import numpy as np
import pandas as pd
import requests

In [ ]:
@dataclass
class DataSetConfig:
    content_path: Path
    embedding_path: Path
    num_of_docs: int
    index_size: int
    bulk_size: int
    index_name: str
    distance: str
    dimension: int
    hnsw_m: int
    hnsw_ef_construction: int
    hnsw_ef: int
    update_docs_per_sec: int

    weaviate_name: str = "benchmark_weaviate"
    weaviate_host: str = "localhost"
    weaviate_port: int = 8091
    weaviate_version: str = "1.24.10"


def get_dataset_config(target_name):
    setting = {
        "100k-768-m32-ef100-ip": {
            "content_path": Path("dataset/passages-c400-jawiki-20230403"),
            "embedding_path": Path("dataset/passages-c400-jawiki-20230403/multilingual-e5-base-passage"),
            "num_of_docs": 5555583,
            "index_size": 100000,
            "bulk_size": 1000,
            "index_name": "Content",
            "distance": "dot",
            "dimension": 768,
            "hnsw_m": 32,
            "hnsw_ef_construction": 200,
            "hnsw_ef": 100,
            "update_docs_per_sec": 0,
        },
        "1m-768-m49-ef100-ip": {
            "content_path": Path("dataset/passages-c400-jawiki-20230403"),
            "embedding_path": Path("dataset/passages-c400-jawiki-20230403/multilingual-e5-base-passage"),
            "num_of_docs": 5555583,
            "index_size": 1000000,
            "bulk_size": 1000,
            "index_name": "Content",
            "distance": "dot",
            "dimension": 768,
            "hnsw_m": 48,
            "hnsw_ef_construction": 200,
            "hnsw_ef": 100,
            "update_docs_per_sec": 0,
        },
        "5m-768-m49-ef100-ip": {
            "content_path": Path("dataset/passages-c400-jawiki-20230403"),
            "embedding_path": Path("dataset/passages-c400-jawiki-20230403/multilingual-e5-base-passage"),
            "num_of_docs": 5555583,
            "index_size": 5000000,
            "bulk_size": 1000,
            "index_name": "Content",
            "distance": "dot",
            "dimension": 768,
            "hnsw_m": 48,
            "hnsw_ef_construction": 200,
            "hnsw_ef": 100,
            "update_docs_per_sec": 0,
        },
    }
    return DataSetConfig(**setting.get(target_name))


In [ ]:
def run_weaviate(config):
    volume_dir = os.getenv("VOLUME_DIR", "./data")
    print(f"Starting {config.weaviate_name}... ", end="")
    docker_cmd = [
        # "sudo",
        "docker", "run", "-d",
        "--name", config.weaviate_name,
        "-p", f"{config.weaviate_port}:8080",
        # "-v", f"{volume_dir}:/data/content",
        f"cr.weaviate.io/semitechnologies/weaviate:{config.weaviate_version}"
    ]
    result = subprocess.run(docker_cmd, capture_output=True, text=True)
    if result.returncode == 0:
        print("[OK]")
    else:
        print(f"[FAIL]")
        print("STDOUT:")
        print(result.stdout)
        print("STDERR:")
        print(result.stderr)


In [ ]:
def stop_weaviate(config):
    print(f"Stopping {config.weaviate_name}... ", end="")
    docker_cmd = [
        # "sudo",
        "docker", "stop", config.weaviate_name
    ]
    result = subprocess.run(docker_cmd, capture_output=True, text=True)
    if result.returncode == 0:
        print("[OK]")
    else:
        print(f"[FAIL]")
        print("STDOUT:")
        print(result.stdout)
        print("STDERR:")
        print(result.stderr)


In [ ]:
def prune_docker(config):
    print(f"Cleaning up... ", end="")
    docker_cmd = [
        # "sudo",
        "docker", "system", "prune", "-f"
    ]
    result = subprocess.run(docker_cmd, capture_output=True, text=True)
    if result.returncode == 0:
        print("[OK]")
    else:
        print(f"[FAIL]")
        print("STDOUT:")
        print(result.stdout)
        print("STDERR:")
        print(result.stderr)


In [ ]:
def print_docker_system_df(config):
    docker_cmd = [
        # "sudo",
        "docker", "system", "df"
    ]
    result = subprocess.run(docker_cmd, capture_output=True, text=True)
    if result.returncode == 0:
        print(result.stdout)
    else:
        print(result.stderr)


In [ ]:
def print_docker_container_stats(config):
    docker_cmd = [
        # "sudo",
        "docker", "container", "stats", "--no-stream"
    ]
    result = subprocess.run(docker_cmd, capture_output=True, text=True)
    containers = {}
    if result.returncode == 0:
        print(result.stdout)
        for line in result.stdout.split("\n"):
            if line.startswith("CONTAINER") or len(line) == 0:
                continue
            values = line.split()
            containers[values[1]] = {
                "container_id": values[0],
                "cpu": values[2],
                "mem": values[6],
                "mem_usage": values[3],
                "mem_limit": values[5],
                "net_in": values[7],
                "net_out": values[9],
                "block_in": values[10],
                "block_out": values[12],
                "pids": values[13],
            }
    else:
        print(result.stderr)
    return containers


In [ ]:
def create_index(config):
    print(F"Creating {config.index_name}... ", end="")
    response = requests.post(f"http://{config.weaviate_host}:{config.weaviate_port}/v1/schema",
                            headers={"Content-Type": "application/json"},
                            json={
                                "class": config.index_name,
                                "vectorIndexType": "hnsw",
                                "vectorIndexConfig": {
                                    "distance": config.distance,
                                    "maxConnections": config.hnsw_m,
                                    "ef": config.hnsw_ef,
                                    "efConstruction": config.hnsw_ef_construction,
                                },
                                "properties": [
                                    {
                                        "name": "doc_id",
                                        "dataType": ["int"]
                                    },
                                    {
                                        "name": "pageId",
                                        "dataType": ["int"]
                                    },
                                    {
                                        "name": "revId",
                                        "dataType": ["int"]
                                    },
                                    {
                                        "name": "section",
                                        "dataType": ["string"],
                                        "indexInverted": True
                                    },
                                    {
                                        "name": "text",
                                        "dataType": ["text"],
                                        "indexInverted": True
                                    },
                                    {
                                        "name": "title",
                                        "dataType": ["text"],
                                        "indexInverted": True
                                    }
                                ]
                            })
    if response.status_code == 200:
        print("[OK]")
    else:
        print(f"[FAIL]\n{response.text}")


In [ ]:
def delete_index(config):
    print(F"Deleting {config.index_name}... ", end="")
    response = requests.delete(f"http://{config.weaviate_host}:{config.weaviate_port}/v1/schema/{config.index_name}")
    if response.status_code == 200:
        print("[OK]")
    else:
        print(f"[FAIL]\n{response.text}")


In [ ]:
def print_indices(config):
    response = requests.post(f"http://{config.weaviate_host}:{config.weaviate_port}/v1/graphql",
                            headers={"Content-Type": "application/json"},
                            json={
                                "query": "{ Aggregate { Content { meta { count } } } }"
                            })
    if response.status_code == 200:
        obj = json.loads(response.text)
        count = obj.get("data").get("Aggregate").get(config.index_name)[0].get("meta").get("count")
        print(f"count: {count}")
        return {
            "num_of_docs": count,
        }
    print("count: FAILED")
    return {}


In [ ]:
def wait_for_weaviate(config, retry_count=60):
    print(f"Waiting for {config.weaviate_name}", end="")
    for i in range(retry_count):
        try:
            response = requests.get(f"http://{config.weaviate_host}:{config.weaviate_port}/v1/nodes")
            if response.status_code == 200:
                obj = json.loads(response.text)
                if obj.get("nodes")[0].get("status") == "HEALTHY":
                    print(" [OK]")        
                    return
        except:
            pass
        print(".", end="")
        time.sleep(1)
    print(" [FAIL]")


In [ ]:
def get_embedding(config, embedding_index, embedding_data, id):
    emb_index = int(id / 100000) * 100000
    if embedding_data is None or embedding_index != emb_index:
        with np.load(config.embedding_path / f"{emb_index}.npz") as data:
            embedding_data = data["embs"]
    embedding = embedding_data[id - emb_index]
    if config.distance == "dot":
        embedding = embedding / np.linalg.norm(embedding)
    return emb_index, embedding_data, embedding


def get_section_values(config, df, min_count=10000):
    tmp_df = df[["id","section"]].groupby("section").count().reset_index()
    tmp_df = tmp_df[tmp_df["id"] >= min_count]
    return tmp_df["section"].values.tolist()


def insert_data(config, max_size, bulk_size, controller=None):
    start_time = time.time()

    docs = []
    def send_data(pos):
        print(F"Sending {int(len(docs))} docs ({pos}/{max_size})... ", end="")
        now = time.time()
        response = requests.post(f"http://{config.weaviate_host}:{config.weaviate_port}/v1/batch/objects",
                                 headers={"Content-Type": "application/json"},
                                 json={
                                     "objects": docs,
                                 })
        if response.status_code == 200:
            t = time.time() - now
            print(f"[OK] {t}")
            return t
        else:
            print(f"[FAIL] 0 {response.status_code} {response.text}")
            return 0

    total_time = 0
    count = 0
    embedding_index = -1
    embedding_data = None
    for content_file in sorted(config.content_path.glob("*.parquet")):
        df = pd.read_parquet(content_file)
        section_values.extend(get_section_values(config, df))
        for i,row in df.iterrows():
            if count >= max_size:
                break
            embedding_index, embedding_data, embedding = get_embedding(config, embedding_index, embedding_data, row.id)
            count += 1
            docs.append({
                "class": config.index_name,
                "properties": {
                    "doc_id": count,
                    "page_id": row.pageid,
                    "rev_id": row.revid,
                    #"title": row.title,
                    "section": row.section,
                    #"text": row.text,
                },
                "vector": embedding.tolist(),                    
            })
            if len(docs) >= bulk_size:
                total_time += send_data(count)
                docs = []
                if controller is not None and not controller.run():
                    return

    if len(docs) > 0:
        total_time += send_data(count)

    execution_time = time.time() - start_time
    hours, remainder = divmod(execution_time, 3600)
    minutes, seconds = divmod(remainder, 60)
    print(f"Execution Time: {int(hours):02d}:{int(minutes):02d}:{seconds:02.2f} ({timedelta(seconds=total_time)})")
    return {
        "execution_time": execution_time,
        "process_time": total_time,
    }


In [ ]:
class IndexingController:

    def __init__(self, stop_event):
        self._previous_time = time.time()
        self._stop_event = stop_event

    def run(self):
        if self._stop_event.is_set():
            return False

        wait_time = 1 - ( time.time() - self._previous_time)
        if wait_time > 0:
            # print(f"Waiting for {wait_time} sec")
            time.sleep(wait_time)
        self._previous_time =  time.time()
        return True


def update_data(target_config, stop_event):
    dataset_config = get_dataset_config(target_config)
    if dataset_config.update_docs_per_sec > 0:
        print(f"Starting update for {target_config}")
        insert_data(dataset_config, 
                    max_size=dataset_config.index_size,
                    bulk_size=dataset_config.update_docs_per_sec,
                    controller=IndexingController(stop_event))
        print(f"Stopping update for {target_config}")
    else:
        print(f"No background updates")


def start_update(target_config):
    stop_event = multiprocessing.Event()

    p = multiprocessing.Process(target=update_data, args=(target_config, stop_event))
    p.start()

    def stop_update():
        stop_event.set()
        p.join()

    return stop_update


In [ ]:
def search(config, query):
    # print(query)
    now = time.time()
    response = requests.post(f"http://{config.weaviate_host}:{config.weaviate_port}/v1/graphql",
                             json={
                                 "query": query
                             })
    took = time.time() - now
    # print(response.text)

    if response.status_code == 200:
        obj = json.loads(response.text)
        results = obj.get("data").get("Get").get(config.index_name)
        product_ids = [x.get("doc_id") for x in results]
        scores = [x.get("_additional").get("distance") for x in results]
        return took * 1000, len(results), product_ids, scores
    print(f"[FAIL][{response.status_code}] {response.text}")
    return -1, -1, [], []


In [ ]:
def create_query(config, embedding, page_size, pre_filter=None):
    where = next(pre_filter) if pre_filter is not None else ""
    return  f"""{{
  Get {{
    {config.index_name} (
      limit: {page_size}
      nearVector: {{
        vector: {json.dumps(embedding)}
      }}
{where}
    ) {{
      doc_id
      section
      _additional {{
        distance
      }}
    }}
  }}
}}"""


def search_with_knn_queries(config, output_path, pre_filter=None, max_size=10000, page_size=100, offset=0, max_error_count=100):
    print("Sending knn queries...")
    start_time = time.time()
    pos = offset
    doc_id = 0
    count = 0
    running = True
    error_count = 0
    with gzip.open(output_path, "wt", encoding="utf-8") as f:
        while running:
            with np.load(config.embedding_path / f"{pos}.npz") as data:
                embedding_data = data["embs"]
            for embedding in embedding_data:
                doc_id += 1
                if count >= max_size:
                    running = False
                    break
                if config.distance == "dot":
                    embedding = embedding / np.linalg.norm(embedding)
                query = create_query(config, embedding.tolist(), page_size, pre_filter)
                # print(query)
                took, hits, ids, scores = search(config, query)
                # print(f"{took}, {ids}, {scores}")
                if took == -1:
                    error_count += 1
                    if error_count >= max_error_count:
                        running = False
                        break
                    continue
                result = {
                    "id": doc_id,
                    "took": took,
                    "hits": hits,
                    "ids": ids,
                    "scores": scores,
                }
                f.write(json.dumps(result, ensure_ascii=False))
                f.write("\n")
                count += 1
                if count % 10000 == 0:
                    print(f"Sent {count}/{max_size} queries.")

            pos += 100000
            if pos > config.num_of_docs:
                pos = 0

    execution_time = time.time() - start_time
    hours, remainder = divmod(execution_time, 3600)
    minutes, seconds = divmod(remainder, 60)
    print(f"Execution Time: {int(hours):02d}:{int(minutes):02d}:{seconds:02.2f}")


In [ ]:
def get_output_filename(version, name, explain=False, track_total_hits=False):
    filename = f"output/weaviate{version.replace('.', '_')}_{name}"
    filename += ".jsonl.gz"
    return filename


In [ ]:
def print_took_and_total_hits(k, filename, truth_filename):
    query_ids = []
    tooks = []
    hits = []
    df = pd.read_json(filename, lines=True)
    result = {
        "num_of_queries": len(df),
        "took": {
            "mean": df.took.mean(),
            "std": df.took.std(),
            "min": df.took.min(),
            "25%": df.took.quantile(0.25),
            "50%": df.took.quantile(0.5),
            "75%": df.took.quantile(0.75),
            "90%": df.took.quantile(0.9),
            "99%": df.took.quantile(0.99),
            "max": df.took.max(),
        },
        "hits": {
            "mean": df.hits.mean(),
            "std": df.hits.std(),
            "min": df.hits.min(),
            "25%": df.hits.quantile(0.25),
            "50%": df.hits.quantile(0.5),
            "75%": df.hits.quantile(0.75),
            "max": df.hits.max(),
        },
    }
    if os.path.exists(truth_filename):
        df = pd.merge(df,
                      pd.read_json(truth_filename, lines=True)[["id", "ids"]].rename(columns={"ids":"truth_ids"}),
                      on="id", how="inner")
        def get_precision(row):
            size = len(row["truth_ids"])
            if size > k:
                size = k
            return len(set(row["ids"]).intersection(set(row["truth_ids"]))) / size
        df["precision"] = df.apply(get_precision, axis=1)
        result["precision"] = {
            "mean": df.precision.mean(),
            "std": df.precision.std(),
            "min": df.precision.min(),
            "25%": df.precision.quantile(0.25),
            "50%": df.precision.quantile(0.5),
            "75%": df.precision.quantile(0.75),
            "90%": df.precision.quantile(0.9),
            "99%": df.precision.quantile(0.99),
            "max": df.precision.max(),
        }
    print(df.describe().to_markdown())
    return result


In [ ]:
def save_results(target_config, config, results):
    with open("results.json", "wt", encoding="utf-8") as f:
        json.dump({
            "target": target_config,
            "version": config.weaviate_version,
            "settings": asdict(config),
            "results": results,
            "timestamp": datetime.now().isoformat(),
        }, f, ensure_ascii=False, default=lambda x: int(x) if isinstance(x, np.int64) else None)


In [ ]:
section_values = []
results = {}
target_config = os.getenv("TARGET_CONFIG", "100k-768-m32-ef100-ip")
dataset_config = get_dataset_config(target_config)
pprint.pprint(dataset_config)

In [ ]:
prune_docker(dataset_config)
print(f"<<<Weaviate {dataset_config.weaviate_version}>>>")
run_weaviate(dataset_config)
wait_for_weaviate(dataset_config)

In [ ]:
print_docker_container_stats(dataset_config)
print_docker_system_df(dataset_config)

In [ ]:
create_index(dataset_config)

In [ ]:
print_docker_container_stats(dataset_config)
print_indices(dataset_config)
print_docker_system_df(dataset_config)

In [ ]:
results["indexing"] = insert_data(dataset_config, max_size=dataset_config.index_size, bulk_size=dataset_config.bulk_size)

In [ ]:
results["indexing"]["container"] = print_docker_container_stats(dataset_config)
print_indices(dataset_config)
print_docker_system_df(dataset_config)

In [ ]:
for page_size in [10, 100, 400]:
    print(f"page size: {page_size}")
    filename = get_output_filename(dataset_config.weaviate_version, f"knn_{page_size}")
    stop_update = start_update(target_config)
    search_with_knn_queries(dataset_config, filename, page_size=page_size, max_size=1000) # warmup
    search_with_knn_queries(dataset_config, filename, page_size=page_size, offset=dataset_config.index_size)
    stop_update()
    results[f"top_{page_size}"] = print_took_and_total_hits(page_size, filename,  f"dataset/ground_truth/{target_config}/knn_{page_size}.jsonl.gz")

In [ ]:
def pre_filter_generator():
    if len(section_values) > 0:
        while True:
            for s in section_values:
                yield f"""
      where: {{
        operator: Equal
        path: ["section"]
        valueString: "{s}"
      }}
"""

results["num_of_filtered_words"] = len(section_values)
for page_size in [10, 100, 400]:
    print(f"page size: {page_size}")
    filename = get_output_filename(dataset_config.weaviate_version, f"knn_{page_size}_filtered", explain=False, track_total_hits=False)
    stop_update = start_update(target_config)
    search_with_knn_queries(dataset_config, filename, page_size=page_size, max_size=1000, pre_filter=pre_filter_generator()) # warmup
    search_with_knn_queries(dataset_config, filename, page_size=page_size, offset=dataset_config.index_size, pre_filter=pre_filter_generator())
    stop_update()
    results[f"top_{page_size}_filtered"] = print_took_and_total_hits(page_size, filename, f"dataset/ground_truth/{target_config}/knn_{page_size}_filtered.jsonl.gz")

In [ ]:
print_docker_container_stats(dataset_config)
print_indices(dataset_config)
print_docker_system_df(dataset_config)

In [ ]:
save_results(target_config, dataset_config, results)

In [ ]:
delete_index(dataset_config)
stop_weaviate(dataset_config)